In [1]:
import tensorflow as tf
import numpy as np

In [11]:
n_inputs = 28 * 28
n_hidden = 100
n_outputs = 5

In [12]:
he_init = tf.contrib.layers.variance_scaling_initializer()
def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.relu, initializer=he_init):
    with tf.variable_scope(name, 'dnn'):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer, name='hidden%d' % (layer + 1))
    return inputs

In [13]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [14]:
dnn_outputs = dnn(X)
logits = tf.layers.dense(dnn_outputs, n_outputs, name='output')

In [15]:
y_proba = tf.nn.softmax(logits, name='probability')

In [16]:
learning_rate = 0.001
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy, name='loss')

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

In [19]:
correct_preds = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32), name='accuracy')

In [20]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/mnist')

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [21]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [23]:
n_epochs = 1000
batch_size = 50

best_score = 0.
max_checks_without_progress = 20
checks_without_progress = 0

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        rnd_indices = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_indices, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_score = sess.run(accuracy, feed_dict={X: X_valid1, y: y_valid1})
        if accuracy_score > best_score:
            print('Epoch %d: Accuracy %.6f' % (epoch, accuracy_score))
            best_score = accuracy_score
            checks_without_progress = 0
            saver.save(sess, './deep_mnist_model.ckpt')
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print('Stopping early!')
                break
    print('Best validation accuracy: %.6f', best_score)
    
with tf.Session() as sess:
    saver.restore(sess, './deep_mnist_model.ckpt')
    accuracy_score = sess.run(accuracy, feed_dict={X: X_test1, y: y_test1})
    print('Test accuracy: %.6f', accuracy_score)

Epoch 0: Accuracy 0.976935
Epoch 1: Accuracy 0.987881
Epoch 3: Accuracy 0.989054
Epoch 5: Accuracy 0.989445
Epoch 6: Accuracy 0.992181
Epoch 11: Accuracy 0.993354
Epoch 27: Accuracy 0.994136
Epoch 33: Accuracy 0.994918
Stopping early!
('Best validation accuracy: %.6f', 0.9949179)
INFO:tensorflow:Restoring parameters from ./deep_mnist_model.ckpt
('Test accuracy: %.6f', 0.99513525)
